# Limpieza LIGIE
## Fases de Tratado
### Fase 0: Configuración General

Antes de iniciar cualquier lógica, es fundamental preparar el entorno de ejecución instalando las dependencias necesarias y comprendiendo las herramientas que manipularán el archivo PDF y las estructuras de datos.

Instalación de dependencias: Para asegurar que el script funcione correctamente, se deben instalar las librerías externas que no vienen incluidas por defecto en Python. Ejecuta el siguiente comando en tu terminal o en una celda de tu entorno de desarrollo:

- pip install pdfplumber pandas numpy xlsxwriter

Librerías:

- pdfplumber: Es el motor principal de extracción. Se elige sobre otras opciones (como PyPDF2) porque permite estrategias visuales (detección de líneas de tablas) y lectura de texto posicional precisa.

- pandas (pd): Utilizada para crear los DataFrames. Es vital para manipular filas, columnas, rellenar datos faltantes (NaN) y transformar texto masivamente.

- numpy (np): Se usa auxiliarmente para definir valores nulos (np.nan) que pandas pueda reconocer y tratar.

- re: Librería de Expresiones Regulares. Es el "cerebro" que identifica patrones de texto complejos (ej. detectar si una línea empieza con "Sección IV" o "Capítulo 85").

- os: Utilidad de sistema operativo para manejo de rutas de archivos.

Variables Globales:

- os.chdir(''): Define el directorio de trabajo.

- PDF_PATH: Define la ruta del archivo origen.

- CODIGO_STOP: El script dejará de leer el PDF cuando encuentre este código específico (9807.00.01), optimizando tiempo al evitar leer anexos irrelevantes al final del documento.

In [1]:
#pip install pdfplumber pandas numpy xlsxwriter

import pdfplumber
import pandas as pd
import numpy as np
import re
import os

#os.chdir('C:/Users/M20537/Downloads')
PDF_PATH = "LIGIE-UNIFICADA-LIGIE_20250728-20250728.pdf"
CODIGO_STOP = "9807.00.01"

### Fase 1: Generación de Catálogos

Esta es la fase más compleja de lógica de texto. Su objetivo es leer el índice inicial del PDF para entender qué significa "Capítulo 01" o "Sección V", resolviendo el problema de que los títulos se cortan entre páginas.

Lectura Masiva:

- En lugar de procesar página por página, el script lee las primeras N páginas (configurado a 25) y extrae todo el texto.

- Acción: Convierte todas las páginas en una lista única y continua de líneas de texto (all_lines).

- Propósito: Eliminar la noción de "salto de página". Para el código, la línea final de la página 3 y la primera de la página 4 son contiguas.

Filtrado de "Basura":

- Se define una lista negra (TEXTOS_BASURA) con encabezados recurrentes ("Secretaría de Economía", "Dirección General...", "Página", etc.).

- El script descarta cualquier línea que contenga estas frases, sea solo numérica (números de página) o sea demasiado corta. Esto limpia el "ruido" visual del PDF.

Búsqueda "Lookahead":

- El script recorre la lista de líneas buscando patrones Regex de Sección [Romano] o Capítulo [Arábigo].

- Al encontrar uno (ej. "Capítulo 41"), busca su descripción.

- Lógica Crítica: Si la descripción no está en la misma línea, el script entra en un bucle que revisa las siguientes 15 líneas. Salta cualquier texto basura hasta encontrar una línea con texto válido (ej. "Pieles y cueros").

- Si encuentra otro Capítulo o Sección antes de encontrar una descripción, se detiene para evitar errores cruzados.

Limpieza de Texto (limpiar_descripcion):

- Una vez hallada la descripción, se eliminan los puntos suspensivos del índice (....) y los números de página finales (390), dejando solo el texto puro ("Pieles y cueros").

Mapeo Jerárquico:

- Se guardan los resultados en diccionarios (dict_capitulos, dict_secciones).

- Se crea un mapa de relación (mapa_cap_sec) que recuerda qué Sección estaba activa cuando se leyó un Capítulo. Esto permite saber después que el Capítulo 01 pertenece a la Sección I.

In [2]:
def generar_catalogos_final(pdf_path, paginas_a_escanear=25):
    print(f"--- FASE 1: Generando catálogos (Escanear primeras {paginas_a_escanear} págs) ---")
    
    dict_capitulos = {}
    dict_secciones = {}
    mapa_cap_sec = {} 
    
    # Regex para capturar el número romano o arábigo
    regex_seccion = re.compile(r"^(?:Sección|SECCIÓN|Seccion)\s+([IVXLCDM]+)", re.IGNORECASE) 
    regex_capitulo = re.compile(r"^(?:Capítulo|CAPÍTULO|Capitulo)\s+(\d+)", re.IGNORECASE)
    
    # Textos "basura" que interrumpen la lectura
    TEXTOS_BASURA = [
        "Calle Pachuca", "www.gob.mx", "Tel:", "(55) 5729", 
        "Economía", "Secretaría de Economía", 
        "Dirección General", "Facilitación", "Comercial y de Comercio Exterior",
        "PISTON MOLLICA", "Versión unificada", "Dudas y/o comentarios",
        "nueva.ligie", "Hoja", "Página", "The following table", 
        "Índice de abreviaturas", "Documento referencial", 
        "Ley de los Impuestos", "7ma Enmienda", "Acuerdo por el que"
    ]

    # 1. Carga masiva de líneas
    all_lines = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            limit = min(paginas_a_escanear, len(pdf.pages))
            for i in range(limit):
                page = pdf.pages[i]
                text = page.extract_text()
                if text:
                    lines = [l.strip() for l in text.split('\n') if l.strip()]
                    all_lines.extend(lines)
    except Exception as e:
        print(f"Error crítico leyendo PDF para catálogos: {e}")
        return {}, {}, {}

    total_lines = len(all_lines)

    # 2. Funciones de limpieza
    def es_linea_util(txt):
        if not txt: return False
        txt_lower = txt.lower()
        if any(basura.lower() in txt_lower for basura in TEXTOS_BASURA): return False
        if re.match(r'^[\d\.\s]+$', txt): return False
        if len(txt) < 3: return False
        return True

    def limpiar_descripcion(txt):
        txt = re.split(r'\.{2,}', txt)[0]
        txt = re.sub(r'\s+\d+$', '', txt)
        txt = txt.lstrip('.- ').strip()
        return txt

    def buscar_descripcion_real(start_index, match_obj):
        # A. Intentar en la misma línea
        parte_match = match_obj.group(0)
        resto_linea = all_lines[start_index][len(parte_match):].strip()
        resto_linea = re.split(r'\.{2,}', resto_linea)[0].strip()
        
        if len(resto_linea) > 2 and not re.match(r'^[\d\.]+$', resto_linea):
            return limpiar_descripcion(resto_linea)
        
        # B. Buscar hacia abajo
        for offset in range(1, 15):
            if start_index + offset >= total_lines: break
            next_line = all_lines[start_index + offset].strip()
            
            if regex_seccion.match(next_line) or regex_capitulo.match(next_line):
                return None 
            
            if es_linea_util(next_line):
                return limpiar_descripcion(next_line)
        return None

    # 3. Procesamiento
    seccion_actual_romana = "ND"

    for idx, line in enumerate(all_lines):
        match_sec = regex_seccion.match(line)
        match_cap = regex_capitulo.match(line)

        if match_sec:
            num = match_sec.group(1).strip()
            seccion_actual_romana = num
            desc = buscar_descripcion_real(idx, match_sec)
            if desc:
                dict_secciones[num] = desc.upper()
        
        elif match_cap:
            num_raw = match_cap.group(1).strip()
            key = f"{int(num_raw):02d}"
            mapa_cap_sec[key] = seccion_actual_romana
            desc = buscar_descripcion_real(idx, match_cap)
            if desc:
                dict_capitulos[key] = desc
            else:
                dict_capitulos[key] = "DESCRIPCIÓN NO ENCONTRADA"

    return dict_capitulos, dict_secciones, mapa_cap_sec

# Ejecutar Fase 1
CAPITULOS_NOMBRES, SECCIONES_NOMBRES, MAPA_CAPITULO_SECCION = generar_catalogos_final(PDF_PATH)
print(f"Catálogos listos: {len(CAPITULOS_NOMBRES)} Capítulos detectados.")


--- FASE 1: Generando catálogos (Escanear primeras 25 págs) ---


KeyboardInterrupt: 

### Fase 2: Extracción de Tablas

Una vez que tenemos los catálogos, pasamos a extraer la información del resto del documento.

Detección de Área de Interés:

- El script itera por cada página buscando la palabra clave "CÓDIGO" (el encabezado de la tabla).

- Usa la coordenada vertical (bottom) de esa palabra para recortar (crop) la página. Todo lo que esté arriba de ese encabezado (logos, títulos de secretaría) es ignorado para no ensuciar la tabla.

Extracción (extract_table):

- Se usa la estrategia lines. pdfplumber busca las líneas negras verticales y horizontales para reconstruir la celda de Excel.

- Se genera un DataFrame temporal por cada página.

Limpieza a Nivel Fila:

- Se eliminan filas que sean repeticiones de encabezados (filas que contienen palabras como "TASA", "IMPUESTO", "ARANCEL").

- Se eliminan filas de paginación o filas vacías que no contienen códigos ni descripciones útiles.

Criterio de Parada:

- En cada página, se verifica si el CODIGO_STOP está presente. Si se encuentra, se cortan los datos hasta esa fila y se detiene el bucle de lectura de páginas completamente.

In [ ]:
all_data = []
COLUMNAS_ESPERADAS = ["CODIGO", "NICO", "DESCRIPCION", "UNIDAD", "IMP_IMP", "IMP_EXP", "ACOTACION"]
PALABRAS_PROHIBIDAS = ["SUBP", "IMPUESTO", "EXP.", "IMP.", "TASA", "CUOTA", "TRATADO", "ARANCEL"]
stop_process_flag = False

print(f"\n--- FASE 2: Extracción de tablas de datos ---")

try:
    with pdfplumber.open(PDF_PATH) as pdf:
        total = len(pdf.pages)
        # Empezamos después del índice para acelerar (aprox pág 13), o desde 0 para seguridad
        start_page = 0 
        
        for i in range(start_page, total):
            if stop_process_flag: break
            page = pdf.pages[i]

            # Buscar encabezado de tabla
            busqueda = page.search("CÓDIGO")
            crop_y = 0
            found_header = False

            if busqueda:
                crop_y = busqueda[0]["bottom"] + 2
                found_header = True

            if found_header:
                try:
                    cropped_page = page.crop((0, crop_y, page.width, page.height))
                    table_settings = {"vertical_strategy": "lines", "horizontal_strategy": "lines", "snap_tolerance": 4}
                    table = cropped_page.extract_table(table_settings)

                    if table:
                        df = pd.DataFrame(table)
                        df = df.dropna(how='all')

                        # Limpiar filas basura repetitivas dentro de la tabla
                        def es_fila_basura(row):
                            texto_fila = " ".join([str(x) for x in row if x is not None]).upper()
                            return any(palabra in texto_fila for palabra in PALABRAS_PROHIBIDAS)

                        while not df.empty and es_fila_basura(df.iloc[0]):
                            df = df.iloc[1:]
                            df.reset_index(drop=True, inplace=True)

                        if df.shape[1] >= 7:
                            conteos = df.count()
                            indices_top = conteos.nlargest(7).index.sort_values()
                            df = df[indices_top]
                            df.columns = COLUMNAS_ESPERADAS

                            mask_basura = (df['CODIGO'].isna() | (df['CODIGO'] == '')) & \
                                          (df['DESCRIPCION'].str.contains("Página", case=False, na=False))
                            df = df[~mask_basura]

                            codigos_str = df['CODIGO'].astype(str).str.strip()
                            if CODIGO_STOP in codigos_str.values:
                                print(f"¡OBJETIVO ENCONTRADO! Código {CODIGO_STOP} detectado en pág {i+1}.")
                                idx_stop = df[codigos_str == CODIGO_STOP].index[0]
                                df = df.iloc[:idx_stop + 1]
                                stop_process_flag = True

                            all_data.append(df)
                except Exception: pass

            if i % 50 == 0: print(f"Procesando página {i+1}/{total}...")

except Exception as e:
    print(f"Error abriendo PDF en Fase 2: {e}")


### Fase 3: Procesamiento y Lógica Jerárquica

Aquí ocurre la transformación de datos crudos a información estructurada.

Unificación y Limpieza Inicial:

- Se concatenan todas las tablas de todas las páginas en un gran DataFrame maestro.

- Se eliminan saltos de línea (\n) dentro de las celdas para que el texto sea continuo.

Lógica de Relleno (Backfill):

- En la LIGIE, a veces aparece la descripción de una Partida (ej. "Caballos") y en la siguiente fila aparece el código desglosado ("0101.21.01"). La fila de "Caballos" tiene el código vacío.

- Acción: El script usa bfill() (Back Fill). Si una celda de código está vacía, mira hacia abajo, toma el código de la siguiente fila (ej. "0101.21.01") y recorta el último par de dígitos. Así, asigna retroactivamente el código padre a la descripción huérfana.

Auto-Aprendizaje de Diccionarios:

- El script barre todo el DataFrame buscando códigos de longitudes específicas para aprender qué es qué:

    - 4 dígitos = Partida.

    - 5 dígitos = Desdoblamiento (Nivel intermedio).

    - 6 dígitos = Subpartida.

- Guarda estas descripciones en nuevos diccionarios para usarlos más tarde en la concatenación.

Filtrado de "Hojas" (Nivel Fracción):

- Se filtra el DataFrame para quedarse únicamente con los registros finales: aquellos que tienen 8 dígitos y una Tasa de Impuesto definida. Esto elimina los encabezados intermedios (que ya guardamos en diccionarios) y deja solo los productos importables.

Desglose y Mapeo (Enriquecimiento):

- Desglose: Se separa el código "01020304" en columnas individuales: Show_Capitulo ("01"), Show_Partida ("02"), Show_Desdoblamiento ("0"), Show_Subpartida ("3"), etc.

- Mapeo de Texto: Usando los diccionarios creados en la Fase 1 y Fase 3, se crea una columna de texto para cada nivel.

    - Ejemplo: La columna Txt_Capitulo busca "01" en el diccionario y devuelve "Animales Vivos.".

    - Homogenización: Se aplica formato de texto: Primera letra mayúscula (Capitalize) y se asegura que todos terminen en punto final.

Concatenación de Texto:

- Se crea la columna Texto_Concatenado. El script toma todos los textos jerárquicos (Sección, Capítulo, Partida, Desdoblamiento, Subpartida, Fracción).

- Une estos textos usando un espacio simple. Al tener puntuación asegurada en el paso anterior, el resultado es una oración fluida y legible.

### Fase 4: Exportación (Excel)

La etapa final es verter los datos procesados en un archivo .xlsx ordenado y formateado.

Creación de Paneles (Sub-tablas):

- Se crean 4 DataFrames derivados con distintas columnas según la necesidad:

    - LIGIE: Datos crudos procesados.

    - Reducido: Enfoque numérico (Códigos desglosados y tasas).

    - Textual: Enfoque descriptivo (Nombres de niveles y tasas).

    - Extendido: La matriz completa (Números + Nombres).

    - Concatenado: Código completo + Descripción unificada.

Motor de Excel (xlsxwriter):

- Se inicia el motor de escritura de Excel.

Estilizado Inteligente:

- Se definen formatos: Encabezados en azul y negrita, bordes en todas las celdas.

- Wrap Text (Ajuste de texto): Se aplica condicionalmente. Si la columna contiene descripciones largas o la columna "ACOTACION", se activa el ajuste de línea para que el texto no se desborde ni se oculte.

Ajuste de Anchos:

- Columnas de códigos (ej. "Capítulo"): Ancho 8 (angosto).

- Columnas de texto (ej. "Descripción"): Ancho 50 (amplio).

- Resto de columnas: Ancho 12 (estándar).

- Escritura: Se guardan las 5 pestañas y se cierra el archivo, finalizando el proceso.

In [ ]:
if all_data:
    print("\n--- FASE 3: Procesamiento y Exportación ---")
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df.replace(r'\n', ' ', regex=True)

    # 1. Relleno Jerárquico
    print("Aplicando relleno de códigos...")
    final_df['NICO'] = final_df['NICO'].fillna('').astype(str).str.strip().replace(['--', '-'], '')
    final_df['CODIGO'] = final_df['CODIGO'].replace(r'^\s*$', np.nan, regex=True)
    
    siguientes_codigos = final_df['CODIGO'].bfill()
    cond_codigo_vacio = final_df['CODIGO'].isna()
    cond_nico_vacio = final_df['NICO'] == ''
    mascara = cond_codigo_vacio & cond_nico_vacio
    
    if not siguientes_codigos[mascara].empty:
        final_df.loc[mascara, 'CODIGO'] = siguientes_codigos[mascara].astype(str).str[:-1]

    final_df = final_df.fillna('')

    # 2. Generación de Diccionarios (Partida, Desdoblamiento, Subpartida)
    print("Extrayendo descripciones jerárquicas...")
    panel_base = final_df.copy()
    panel_base['CODIGO_LIMPIO'] = panel_base['CODIGO'].astype(str).str.replace('.', '', regex=False).str.strip()

    dict_partidas = {}       # 4 dígitos
    dict_desdoblamiento = {} # 5 dígitos
    dict_subpartidas = {}    # 6 dígitos

    for row in panel_base.itertuples():
        code = getattr(row, 'CODIGO_LIMPIO', '')
        desc = getattr(row, 'DESCRIPCION', '')
        
        if len(code) == 4:
            dict_partidas[code] = desc
        elif len(code) == 5:
            dict_desdoblamiento[code] = desc
        elif len(code) == 6:
            dict_subpartidas[code] = desc

    # 3. Filtrado Final (Solo filas de producto final)
    panel_df = panel_base[
        (panel_base['CODIGO_LIMPIO'].str.len() == 8) &
        (panel_base['IMP_IMP'].str.strip() != '')
    ].copy()
    
    panel_df = panel_df.drop_duplicates(subset=['CODIGO', 'NICO'], keep='first')

    # 4. Desglose de Dígitos (Columnas numéricas "Show_")
    print("Desglosando dígitos...")
    code_full = panel_df['CODIGO_LIMPIO']
    
    panel_df['Show_Capitulo']       = code_full.str[:2]
    panel_df['Show_Partida']        = code_full.str[2:4]
    panel_df['Show_Desdoblamiento'] = code_full.str[4:5]
    panel_df['Show_Subpartida']     = code_full.str[5:6]
    panel_df['Show_Fraccion']       = code_full.str[6:8]

    # 5. Mapeo de Textos
    def get_seccion_romana(cap_code):
        return MAPA_CAPITULO_SECCION.get(cap_code, "ND")

    panel_df['Txt_Seccion_Romana'] = code_full.str[:2].apply(get_seccion_romana)
    
    panel_df['Txt_Seccion_Nombre'] = panel_df['Txt_Seccion_Romana'].map(SECCIONES_NOMBRES)
    panel_df['Txt_Capitulo']       = code_full.str[:2].map(CAPITULOS_NOMBRES)
    panel_df['Txt_Partida']        = code_full.str[:4].map(dict_partidas)
    panel_df['Txt_Desdoblamiento'] = code_full.str[:5].map(dict_desdoblamiento)
    panel_df['Txt_Subpartida']     = code_full.str[:6].map(dict_subpartidas)

    # 6. Homogenización de Formatos de Texto
    print("Formateando textos (Mayúsculas y Puntos)...")
    
    # Sección: Capitalize + Punto
    panel_df['Txt_Seccion_Nombre'] = panel_df['Txt_Seccion_Nombre'].fillna('').astype(str).str.lower().str.capitalize()
    panel_df['Txt_Seccion_Nombre'] = panel_df['Txt_Seccion_Nombre'].str.rstrip('.') + '.'
    
    # Capítulo: Asegurar punto final
    panel_df['Txt_Capitulo'] = panel_df['Txt_Capitulo'].fillna('').astype(str).str.rstrip('.') + '.'

    panel_df['Codigo_Completo'] = panel_df['CODIGO']

    # 7. Concatenación (Sin PIPES, solo espacio)
    print("Generando concatenación limpia...")
    
    def concatenar_descripciones(row):
        items = [
            row['Txt_Seccion_Nombre'],
            row['Txt_Capitulo'],
            row['Txt_Partida'],
            row['Txt_Desdoblamiento'],
            row['Txt_Subpartida'],
            row['DESCRIPCION']
        ]
        # Unir con espacio simple. Filtra vacíos y 'nan'.
        return " ".join([str(x).strip() for x in items if str(x).strip() != '' and str(x).strip() != 'nan'])

    panel_df['Texto_Concatenado'] = panel_df.apply(concatenar_descripciones, axis=1)

    # --- DEFINICIÓN DE ESTRUCTURAS DE SALIDA ---

    # Panel Reducido
    cols_reducido = [
        'Codigo_Completo', 'Txt_Seccion_Romana', 
        'Show_Capitulo', 'Show_Partida', 'Show_Desdoblamiento', 'Show_Subpartida', 'Show_Fraccion', 
        'UNIDAD', 'IMP_IMP', 'IMP_EXP'
    ]
    df_reducido = panel_df[cols_reducido].copy()
    df_reducido.columns = [
        'Código Completo', 'Sección', 
        'Capítulo', 'Partida', 'Desdoblamiento', 'Subpartida', 'Fracción', 
        'Unidad', 'Imp. Imp.', 'Imp. Exp.'
    ]

    # Panel Textual
    cols_textual = [
        'Codigo_Completo', 
        'Txt_Seccion_Nombre', 'Txt_Capitulo', 'Txt_Partida', 
        'Txt_Desdoblamiento', 'Txt_Subpartida', 
        'DESCRIPCION', 
        'UNIDAD', 'IMP_IMP', 'IMP_EXP'
    ]
    df_textual = panel_df[cols_textual].copy()
    df_textual.columns = [
        'Código Completo', 
        'Nombre Sección', 'Nombre Capítulo', 'Nombre Partida', 
        'Nombre Desdoblamiento', 'Nombre Subpartida', 
        'Descripción Fracción', 
        'Unidad', 'Imp. Imp.', 'Imp. Exp.'
    ]

    # Panel Extendido
    cols_extendido = [
        'Codigo_Completo', 
        'Txt_Seccion_Romana', 'Txt_Seccion_Nombre',
        'Show_Capitulo', 'Txt_Capitulo',
        'Show_Partida', 'Txt_Partida',
        'Show_Desdoblamiento', 'Txt_Desdoblamiento', 
        'Show_Subpartida', 'Txt_Subpartida',
        'Show_Fraccion', 
        'DESCRIPCION',   
        'UNIDAD', 'IMP_IMP', 'IMP_EXP'
    ]
    df_extendido = panel_df[cols_extendido].copy()
    df_extendido.columns = [
        'Código Completo', 
        'Sección', 'Nombre Sección', 
        'Capítulo', 'Nombre Capítulo',
        'Partida', 'Nombre Partida',
        'Desdob.', 'Nombre Desdoblamiento',
        'Subpartida', 'Nombre Subpartida',
        'Fracción', 
        'Descripción Fracción',
        'Unidad', 'Imp. Imp.', 'Imp. Exp.'
    ]

    # Panel Concatenado
    cols_concat = ['Codigo_Completo', 'Texto_Concatenado', 'UNIDAD', 'IMP_IMP', 'IMP_EXP']
    df_concatenado = panel_df[cols_concat].copy()
    df_concatenado.columns = ['Código Completo', 'Descripción Completa Concatenada', 'Unidad', 'Imp. Imp.', 'Imp. Exp.']

    # --- EXPORTACIÓN A EXCEL ---
    nombre_salida = "LIGIE_Maestra_Unificada.xlsx"
    print(f"Generando archivo Excel: {nombre_salida} ...")

    writer = pd.ExcelWriter(nombre_salida, engine='xlsxwriter')
    workbook = writer.book

    # Formatos
    fmt_header = workbook.add_format({'bold': True, 'border': 1, 'bg_color': '#D9E1F2', 'valign': 'top', 'align': 'center', 'font_name': 'Arial', 'font_size': 10})
    fmt_normal = workbook.add_format({'border': 1, 'valign': 'top', 'font_name': 'Arial', 'font_size': 9})
    fmt_wrap = workbook.add_format({'border': 1, 'valign': 'top', 'text_wrap': True, 'font_name': 'Arial', 'font_size': 9})

    def escribir_hoja(dataframe, nombre_hoja):
        worksheet = workbook.add_worksheet(nombre_hoja)
        writer.sheets[nombre_hoja] = worksheet
        
        headers = dataframe.columns.tolist()
        for col_idx, header in enumerate(headers):
            worksheet.write(0, col_idx, header, fmt_header)
            
        for row_idx, row in enumerate(dataframe.itertuples(index=False), start=1):
            for col_idx, value in enumerate(row):
                header_name = headers[col_idx]
                
                # Columnas con Wrap Text: Descripciones, Nombres, Acotación y Concatenada
                columnas_largas = ["Nombre", "DESCRIPCION", "Descripción", "ACOTACION", "Concatenada"]
                es_texto_largo = any(x in header_name for x in columnas_largas)
                
                formato = fmt_wrap if es_texto_largo else fmt_normal
                if pd.isna(value): value = ""
                worksheet.write(row_idx, col_idx, value, formato)
        
        # Ajuste inteligente de anchos
        for idx, col_name in enumerate(headers):
            # Columnas de códigos cortos
            if col_name in ['Capítulo', 'Partida', 'Desdob.', 'Desdoblamiento', 'Subpartida', 'Fracción', 'Sección']:
                 worksheet.set_column(idx, idx, 8) 
            # Columnas de texto largo
            elif any(x in col_name for x in ["Nombre", "DESCRIPCION", "Descripción", "ACOTACION", "Concatenada"]):
                worksheet.set_column(idx, idx, 50) 
            # Resto (Unidad, Tasas, Código Completo)
            else:
                worksheet.set_column(idx, idx, 12)

    # Escritura de Hojas
    escribir_hoja(final_df, "LIGIE") 
    escribir_hoja(df_reducido, "Panel Reducido")
    escribir_hoja(df_textual, "Panel Textual")
    escribir_hoja(df_extendido, "Panel Extendido")
    escribir_hoja(df_concatenado, "Concatenado")

    writer.close()
    print("¡PROCESO FINALIZADO CON ÉXITO!")

else:
    print("No se encontraron datos para exportar. Verifica el archivo PDF.")